In [11]:
import pandas as pd
from google.cloud import storage
from pandas_gbq import to_gbq

In [2]:
df = pd.read_csv('Marvel_Comics.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34466 non-null  object
 3   publish_date       34466 non-null  object
 4   issue_description  30395 non-null  object
 5   penciler           25482 non-null  object
 6   writer             27595 non-null  object
 7   cover_artist       12255 non-null  object
 8   Imprint            11684 non-null  object
 9   Format             32894 non-null  object
 10  Rating             12619 non-null  object
 11  Price              32894 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [4]:
df['Rating'].value_counts()

Rating
Rated T+                              3204
T                                     1564
Rated T                               1407
Parental Advisory                     1205
T+                                    1000
All Ages                               893
A                                      635
Marvel Psr                             534
MARVEL PSR                             526
ALL AGES                               304
PARENTAL ADVISORY                      207
RATED T+                               206
Rated a                                135
EXPLICIT CONTENT                       110
RATED A                                105
No Rating                              100
RATED T                                 86
PARENTAL ADVISORY/EXPLICIT CONTENT      70
Parental Advisory/Explicit Content      69
Mature                                  44
Explicit Content                        39
Max                                     28
Rated A                                 28
PARE

In [5]:
# Loop tiap kolom dan isi Null values
for col in df.columns:
    if df[col].dtype == 'object':  # hanya kolom bertipe string/object
        df[col] = df[col].fillna('Not Provided')
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34992 non-null  object
 3   publish_date       34992 non-null  object
 4   issue_description  34992 non-null  object
 5   penciler           34992 non-null  object
 6   writer             34992 non-null  object
 7   cover_artist       34992 non-null  object
 8   Imprint            34992 non-null  object
 9   Format             34992 non-null  object
 10  Rating             34992 non-null  object
 11  Price              34992 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [7]:
# Simpan hasilnya
df.to_csv('data_cleaned.csv', index=False)

In [10]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_path):
    # Inisialisasi client GCS
    client = storage.Client()
    
    # Ambil bucket-nya
    bucket = client.bucket(bucket_name)
    
    # Tentukan blob tujuan
    blob = bucket.blob(destination_blob_path)
    
    # Upload file dari lokal
    blob.upload_from_filename(source_file_path)

    print(f"File '{source_file_path}' berhasil diupload ke 'gs://{bucket_name}/{destination_blob_path}'")
    
upload_to_gcs(
    bucket_name="sandbox-buckets-trial",
    source_file_path="data_cleaned.csv",
    destination_blob_path="processed/data_cleaned.csv"
)

File 'data_cleaned.csv' berhasil diupload ke 'gs://sandbox-buckets-trial/processed/data_cleaned.csv'


In [12]:
# Set nama tabel tujuan di BigQuery
dataset_id = "sandbox"
table_id = f"{dataset_id}.marvel_comics"   # tanpa project ID di sini
project_id = "poc-mulia-ceramics-ai"

# Upload ke BigQuery
to_gbq(
    dataframe=df,
    destination_table=table_id,
    project_id=project_id,
    if_exists="replace"  # opsi: 'fail', 'replace', 'append'
)

print("Data berhasil diupload ke BigQuery!")

100%|██████████| 1/1 [00:00<00:00, 7157.52it/s]

Data berhasil diupload ke BigQuery!
